# Added Mass Motions

In this notebook you will find:

1. An implementation of the equations of motion for added mass found in Fossen's _Guidance and Control of Ocean Vehicles_ (1994).

2. Integration of the equations of motion using a Semi-implicit Euler integrator.

3. Plots from the integrated motion.

---

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

import quaternion as qt

import scipy as sp

---

# 1. Implementation of the Equations of Motion

In [ ]:
# Custom `dtype` used to store poses. Each pose will contain a timestamp 't', a position 'p', and a
# quaternion representing orientation 'q'.
pose_dtype = np.dtype([('t', np.float64), ('p', np.float64, 3), ('q', qt.quaternion)])

In [ ]:
def hat(v):
    """Return the left cross-product matrix for the input vector."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

In [ ]:
def generate_equations_of_motion(spatial_inertia_body):
    """
    Return a function implementing the equations of motion of the rigid body in a fixed frame.
    
    The implementation follows the derivation in Fossen's _Guidance and Control of Ocean Vehicles_ (1994).
    
    Args:
        spatial_inertia_body: The 6×6 (body-fixed) spatial inertia matrix, expressed in the body frame. It
            must be invertible, but no other assumptions are made (e.g. no assumptions about symmetry or
            positive-definiteness).
    
    Returns:
        A function implementing the equations of motion of the rigid body in a fixed frame. The function takes
        three arguments: `pose`, a scalar ndarray of dtype `pose_dtype`, with the body's pose; `vels`,
        a 6-element 1-dimensional ndarray, with the body's linear and angular velocities, in this order; and
        `forces`, a 6-element 1-dimensional ndarray, with the applied forces and torques, in this order. The
        function returns a 6-element 1-dimensional ndarray, containing the resulting linear and angular
        accelerations, in this order. It requires all quantities to be expressed in the same fixed frame.
    """
    
    # Implementation of the rigid body equations of motion in the body frame. The implementation corresponds
    # to equation (2.90) in Fossen's book, and follows the same nomenclature.
    def eomB(M, ν, τ):
        ν1 = ν[:3]
        ν2 = ν[3:]
        M11 = M[:3, :3]
        M12 = M[:3, 3:]
        M21 = M[3:, :3]
        M22 = M[3:, 3:]
        C = np.r_[
            np.c_[np.zeros((3, 3)), -hat(M11 @ ν1 + M12 @ ν2)],
            np.c_[-hat(M11 @ ν1 + M12 @ ν2), -hat(M21 @ ν1 + M22 @ ν2)]]
        return np.linalg.inv(M) @ (τ - C @ ν)

    # Implementation of the rigid body equations of motion in the fixed frame.
    def eom(pose, vels, forces):
        R = qt.as_rotation_matrix(pose['q'])
        RR = sp.linalg.block_diag(R, R)
        M = RR @ spatial_inertia_body @ RR.T
        w = vels[3:]
        ν = vels
        τ = forces
        νd = eomB(M, ν, τ)
        return νd + sp.linalg.block_diag(hat(w), hat(w)) @ vels
    
    return eom

---

# 2. Integration of the Equations of Motion

## 2.1 Discretization of the Equations of Motion

In [ ]:
def generate_discrete_eom(eom, Δt):
    """
    Given a continuous version of the equations of motion, return a discretized version.
    
    This implementation uses Semi-implicit Euler to discretize the equations of motion.
    
    Args:
        eom: The continuous version of the equations of motion.
        Δt: Timestep used to discretize the equations of motion.
    
    Returns:
        A function that implements a discretized version of the equations of motion. The function takes 6
        arguments, the first 3 acting as arguments and the remaining 3 acting as outputs. The input arguments
        are: `pose`, a scalar ndarray of dtype `pose_dtype`, with the body's pose before the application of
        the equations of motion; `vels`, a 6-element 1-dimensional ndarray, with the body's linear and angular
        velocities, in this order, before the application of the equations of motion; and `forces`, a
        6-element 1-dimensional ndarray, with the applied forces and torques, in this order, before the
        application of the equations of motion. The output arguments are: `pose_out`, a scalar ndarray of
        type `pose_dtype`, with the body's pose after the application of the equations of motion;
        `vels_out`, a 6-element 1-dimensional ndarray, with the body's linear and angular velocities, in this
        order, after the application of the equations of motion; and `accs_out`, a 6-element 1-dimensional
        ndarray, with the linear and angular accelerations, in this order, returned by the continuous
        equations of motion. The function has no return values.
    """
    
    def discrete_eom(pose, vels, forces, pose_out, vels_out, accs_out):
        accs_out[:] = eom(pose, vels, forces)
        vels_out[:] = vels + Δt * accs_out
        pose_out['t'] = pose['t'] + Δt
        pose_out['p'] = pose['p'] + Δt * vels_out[:3]
        pose_out['q'] = qt.from_rotation_vector(Δt * vels_out[3:]) * pose['q']
    
    return discrete_eom

---

## 2.2 Integrate the Equations of Motion

In [ ]:
def apply_discrete_eom(pose0, vels0, forces, discrete_eom, n_steps):
    """
    Apply the equations of motion for a given number of time steps.
    
    Args:
        pose0: Initial pose. This must be a scalar ndarray with dtype `pose_dtype`.
        vels0: Initial linear and angular velocity. This must be a 6-element 1-dimensional
            ndarray, the first 3 elements being the linear velocity and the remaining 3 elements being the
            angular velocity.
        forces: Forces and torques, applied during the motion. This must be a `(n_steps, 6)`-shaped ndarray,
            with each row corresponding to the forces and torques applied at each timestep. The first three
            columns correspond to forces and the remaining three columns correspond to torques.
        discrete_eom: Discrete version of the equations of motion. This matches the signature of the return
            value of `generate_discrete_eom`.
        n_steps: Number of timesteps to apply the equations of motion.

    Returns:
        A tuple of ndarrays: The first ndarray is `(n_steps + 1)`-shaped with dtype `pose_dtype`; the n-th
        column contains the pose before the n-th step. The second ndarray is `(n_steps + 1, 6)`-shaped; the
        n-th column contains the linear and angular velocities, in this order, before the n-th timestep. The
        third (and last) ndarray is `(n_steps, 6)`-shaped; the n-th column contains the linear and angular
        acceleration, in this order, before the n-th timestep.
    """
    poses = np.zeros(n_steps + 1, dtype=pose_dtype)
    vels = np.zeros((n_steps + 1, 6))
    accs = np.zeros((n_steps, 6))
    
    poses[0] = pose0
    vels[0] = vels0
    
    for n in range(n_steps):
        discrete_eom(poses[n], vels[n], forces[n], poses[n + 1], vels[n + 1], accs[n])
    
    return poses, vels, accs

---

In [ ]:
# Body spatial inertia (in the body frame). Top-left quarter corresponds to body mass, bottom-right quarter is
# the tensor of inertia.
spatial_inertia_body = np.eye(6)

In [ ]:
# Equations of motion.
eom = generate_equations_of_motion(spatial_inertia_body)

In [ ]:
# Integration timestep.
Δt = 1e-3

In [ ]:
# Discrete equations of motion.
discrete_eom = generate_discrete_eom(eom, Δt)

In [ ]:
# Initial pose.
pose0 = np.zeros((), dtype=pose_dtype)
pose0['t'] = 0
pose0['p'] = np.zeros(3)
pose0['q'] = qt.one

In [ ]:
# Initial linear and angular velocities.
vels0 = np.zeros(6)

In [ ]:
# Number of integration steps.
n_steps = int(1e3) - 1

In [ ]:
# Forces and torques applied throughout the motion, in the fixed reference frame.
forces = np.zeros((n_steps, 6))

In [ ]:
poses, vels, accs = apply_discrete_eom(pose0, vels0, forces, discrete_eom, n_steps)

---

# 3. Plot the Integrated Motion

In [ ]:
# Position

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(poses['t'], poses['p'][:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(poses['t'], poses['p'][:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(poses['t'], poses['p'][:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Position')

In [ ]:
# Orientation

fig = plt.figure()

plt.subplot(2, 2, 1)
plt.plot(poses['t'], qt.as_float_array(poses['q'])[:, 0])
plt.grid()
plt.ylabel('w')

plt.subplot(2, 2, 2)
plt.plot(poses['t'], qt.as_float_array(poses['q'])[:, 1])
plt.grid()
plt.ylabel('x')

plt.subplot(2, 2, 3)
plt.plot(poses['t'], qt.as_float_array(poses['q'])[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('y')

plt.subplot(2, 2, 4)
plt.plot(poses['t'], qt.as_float_array(poses['q'])[:, 3])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Quaternion')

In [ ]:
# Linear Velocity

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(poses['t'], vels[:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(poses['t'], vels[:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(poses['t'], vels[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Linear Velocity')

In [ ]:
# Angular Velocity

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(poses['t'], vels[:, 3])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(poses['t'], vels[:, 4])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(poses['t'], vels[:, 5])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Angular Velocity')

In [ ]:
# Linear Acceleration

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(poses['t'][:-1], accs[:, 0])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(poses['t'][:-1], accs[:, 1])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(poses['t'][:-1], accs[:, 2])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Linear Acceleration')

In [ ]:
# Angular Acceleration

fig = plt.figure()

plt.subplot(3, 1, 1)
plt.plot(poses['t'][:-1], accs[:, 3])
plt.grid()
plt.ylabel('x')

plt.subplot(3, 1, 2)
plt.plot(poses['t'][:-1], accs[:, 4])
plt.grid()
plt.ylabel('y')

plt.subplot(3, 1, 3)
plt.plot(poses['t'][:-1], accs[:, 5])
plt.grid()
plt.xlabel('time')
plt.ylabel('z')

fig.suptitle('Angular Acceleration')